# NewsAPI

In [ ]:
import requests

api_key = 'API_GENERADA:PAG_WEB'
url = f'https://gnews.io/api/v4/top-headlines?country=pe&category=general&apikey={api_key}'

response = requests.get(url)
data = response.json()

# Convertir a DataFrame
import pandas as pd
df = pd.DataFrame(data['articles'])
print(df.head())



                                               title  \
0  Día Mundial de la Actividad Física: Minsa recu...   
1  Las fuertes imágenes del choque de mototaxi qu...   
2  Cantidad de días no laborables disminuirá a la...   
3  Samsung presentó en Colombia la nueva serie Ga...   
4  Q‑Symphony: integración inteligente entre TV y...   

                                         description  \
0  “Es tiempo de moverse”. Este es el lema del 20...   
1  Mototaxi funcionaba como movilidad escolar y l...   
2  Para este 2025 el Gobierno solo ha establecido...   
3  Las herramientas permiten una interacción más ...   
4  Descubre cómo la función exclusiva de Samsung ...   

                                             content  \
0  “Es tiempo de moverse”. Este es el lema del 20...   
1  Un trágico accidente ocurrió la tarde del pasa...   
2  Así, determinó dos días no laborables para est...   
3  Ingrese o regístrese acá para guardar los artí...   
4  Descubre cómo la función exclusiva de Samsu

In [16]:
df.head()

,title,description,content,url,image,publishedAt,source
0,Día Mundial de la Actividad Física: Minsa recu...,“Es tiempo de moverse”. Este es el lema del 20...,“Es tiempo de moverse”. Este es el lema del 20...,https://www.gob.pe/institucion/minsa/noticias/...,https://cdn.www.gob.pe/uploads/document/file/7...,2025-04-04T20:48:48Z,"{'name': 'gob.pe', 'url': 'https://www.gob.pe'}"
1,Las fuertes imágenes del choque de mototaxi qu...,Mototaxi funcionaba como movilidad escolar y l...,Un trágico accidente ocurrió la tarde del pasa...,https://trome.com/actualidad/policiales/mancha...,https://trome.com/resizer/v2/D2RG6XNKDNFMXOFRX...,2025-04-04T18:51:18Z,"{'name': 'Trome.com', 'url': 'https://trome.com'}"
2,Cantidad de días no laborables disminuirá a la...,Para este 2025 el Gobierno solo ha establecido...,"Así, determinó dos días no laborables para est...",https://gestion.pe/economia/management-empleo/...,https://gestion.pe/resizer/v2/LKOVHT5BSNDWRGNK...,2025-04-04T18:46:43Z,"{'name': 'Gestión', 'url': 'https://gestion.pe'}"
3,Samsung presentó en Colombia la nueva serie Ga...,Las herramientas permiten una interacción más ...,Ingrese o regístrese acá para guardar los artí...,https://www.eltiempo.com/tecnosfera/dispositiv...,https://imagenes.eltiempo.com/files/og_thumbna...,2025-04-04T18:38:57Z,"{'name': 'ELTIEMPO.COM', 'url': 'https://www.e..."
4,Q‑Symphony: integración inteligente entre TV y...,Descubre cómo la función exclusiva de Samsung ...,Descubre cómo la función exclusiva de Samsung ...,https://news.samsung.com/cl/q%E2%80%91symphony...,https://img.global.news.samsung.com/cl/wp-cont...,2025-04-04T18:02:29Z,"{'name': 'Samsung Newsroom', 'url': 'https://n..."


In [21]:
import pandas as pd
import numpy as np
from datetime import datetime

def evaluar_calidad_noticias(df):
    metricas = {}

    # --------------------------
    # 1. Completitud
    completitud = df.notnull().mean()
    for col in df.columns:
        metricas[f"completitud_{col} (%)"] = completitud[col]

    # --------------------------
    # 2. Unicidad
    metricas["duplicados_exactos (%)"] = df.drop(columns=['source']).duplicated().mean()

    metricas["duplicados_titulo (%)"] = df['title'].duplicated().mean()

    # --------------------------
    # 3. Validez
    # Fecha válida
    try:
        df['publishedAt'] = pd.to_datetime(df['publishedAt'], errors='coerce')
        metricas["fecha_valida (%)"] = df['publishedAt'].notnull().mean()
    except:
        metricas["fecha_valida (%)"] = 0

    # URL válida (simple validación)
    metricas["url_valida (%)"] = df['url'].apply(lambda x: isinstance(x, str) and x.startswith("http")).mean()
    metricas["image_url_valida (%)"] = df['image'].apply(lambda x: isinstance(x, str) and x.startswith("http")).mean()

    # --------------------------
    # 4. Consistencia
    metricas["contenido_distinto_de_descripcion (%)"] = (df["content"] != df["description"]).mean()

    # --------------------------
    # 5. Conformidad
    metricas["source_tipo_diccionario (%)"] = df['source'].apply(lambda x: isinstance(x, dict)).mean()

    # --------------------------
    # 6. Timeliness (actualización)
    # 6. Timeliness (actualización)
    if df['publishedAt'].notnull().any():
        ultima_fecha = df['publishedAt'].max().tz_localize(None)
        hoy = pd.Timestamp.today()
        metricas["dias_desde_ultima_noticia"] = (hoy - ultima_fecha).days
    else:
        metricas["dias_desde_ultima_noticia"] = np.nan


    # --------------------------
    # 7. Precisión del contenido (textos no vacíos)
    metricas["titulo_no_vacio (%)"] = df['title'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 10).mean()
    metricas["contenido_no_vacio (%)"] = df['content'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 30).mean()

    # --------------------------
    # Mostrar métricas
    for k, v in metricas.items():
        print(f"{k}: {round(v * 100, 2) if isinstance(v, float) else v}")

    return metricas


In [22]:
evaluar_calidad_noticias(df)

completitud_title (%): 100.0
completitud_description (%): 100.0
completitud_content (%): 100.0
completitud_url (%): 100.0
completitud_image (%): 100.0
completitud_publishedAt (%): 100.0
completitud_source (%): 100.0
duplicados_exactos (%): 0.0
duplicados_titulo (%): 0.0
fecha_valida (%): 100.0
url_valida (%): 100.0
image_url_valida (%): 100.0
contenido_distinto_de_descripcion (%): 100.0
source_tipo_diccionario (%): 100.0
dias_desde_ultima_noticia: -1
titulo_no_vacio (%): 100.0
contenido_no_vacio (%): 100.0


{'completitud_title (%)': np.float64(1.0),
 'completitud_description (%)': np.float64(1.0),
 'completitud_content (%)': np.float64(1.0),
 'completitud_url (%)': np.float64(1.0),
 'completitud_image (%)': np.float64(1.0),
 'completitud_publishedAt (%)': np.float64(1.0),
 'completitud_source (%)': np.float64(1.0),
 'duplicados_exactos (%)': np.float64(0.0),
 'duplicados_titulo (%)': np.float64(0.0),
 'fecha_valida (%)': np.float64(1.0),
 'url_valida (%)': np.float64(1.0),
 'image_url_valida (%)': np.float64(1.0),
 'contenido_distinto_de_descripcion (%)': np.float64(1.0),
 'source_tipo_diccionario (%)': np.float64(1.0),
 'dias_desde_ultima_noticia': -1,
 'titulo_no_vacio (%)': np.float64(1.0),
 'contenido_no_vacio (%)': np.float64(1.0)}

# api de temperatura

In [1]:
import requests
import pandas as pd
from datetime import date

# Lista de ciudades y coordenadas
ciudades = {
    'Lima': (-12.0464, -77.0428),
    'Arequipa': (-16.4090, -71.5375),
    'Cusco': (-13.5319, -71.9675),
    'Piura': (-5.1945, -80.6328),
    'Puno': (-15.8402, -70.0219),
}

# Fechas de interés
fecha_inicio = '2023-12-01'
fecha_fin = '2024-12-31'

# DataFrame acumulativo
df_total = pd.DataFrame()

# Extracción por ciudad
for ciudad, (lat, lon) in ciudades.items():
    url = (
        f'https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}'
        f'&start_date={fecha_inicio}&end_date={fecha_fin}&daily=temperature_2m_max,temperature_2m_min'
        f'&timezone=America%2FLima'
    )
    response = requests.get(url)
    data = response.json()
    
    if 'daily' in data:
        df = pd.DataFrame(data['daily'])
        df['ciudad'] = ciudad
        df_total = pd.concat([df_total, df], ignore_index=True)
    else:
        print(f"⚠️ No se obtuvo data para {ciudad}")

# Mostrar una muestra del DataFrame
print(df_total.head())


         time  temperature_2m_max  temperature_2m_min ciudad
0  2023-12-01                21.9                17.7   Lima
1  2023-12-02                22.3                17.7   Lima
2  2023-12-03                21.5                17.1   Lima
3  2023-12-04                21.3                17.3   Lima
4  2023-12-05                21.8                17.4   Lima


In [2]:
df_total

,time,temperature_2m_max,temperature_2m_min,ciudad
0,2023-12-01,21.9,17.7,Lima
1,2023-12-02,22.3,17.7,Lima
2,2023-12-03,21.5,17.1,Lima
3,2023-12-04,21.3,17.3,Lima
4,2023-12-05,21.8,17.4,Lima
...,...,...,...,...
1980,2024-12-27,15.5,3.7,Puno
1981,2024-12-28,15.4,7.0,Puno
1982,2024-12-29,16.4,6.8,Puno
1983,2024-12-30,16.5,5.7,Puno


In [3]:
# 1. Completitud
print("\n📌 Completitud (nulos):")
faltantes = df_total.isnull().sum()
print(faltantes)

# 2. Exactitud (rango razonable de temperatura)
print("\n📌 Exactitud:")
fuera_de_rango = df_total[
    (df_total['temperature_2m_max'] > 60) | 
    (df_total['temperature_2m_max'] < -30) |
    (df_total['temperature_2m_min'] > 60) | 
    (df_total['temperature_2m_min'] < -30)
]
print(f"Registros fuera de rango: {len(fuera_de_rango)}")

# 3. Consistencia
print("\n📌 Consistencia:")
inconsistencias = df_total[df_total['temperature_2m_min'] > df_total['temperature_2m_max']]
print(f"Registros con inconsistencia temperatura min > max: {len(inconsistencias)}")

# 4. Unicidad
print("\n📌 Unicidad:")
duplicados = df_total.duplicated(subset=['time', 'ciudad'])
print(f"Registros duplicados (fecha + ciudad): {duplicados.sum()}")

# 5. Actualización (ver si la última fecha está presente)
print("\n📌 Actualización:")
if fecha_fin in df_total['time'].values:
    print("✅ Última fecha está presente.")
else:
    print("❌ Faltan registros recientes.")

# 6. Validez
print("\n📌 Validez:")
tipos = df_total.dtypes
print(tipos)



📌 Completitud (nulos):
time                  0
temperature_2m_max    0
temperature_2m_min    0
ciudad                0
dtype: int64

📌 Exactitud:
Registros fuera de rango: 0

📌 Consistencia:
Registros con inconsistencia temperatura min > max: 0

📌 Unicidad:
Registros duplicados (fecha + ciudad): 0

📌 Actualización:
✅ Última fecha está presente.

📌 Validez:
time                   object
temperature_2m_max    float64
temperature_2m_min    float64
ciudad                 object
dtype: object


## api de tipo de cambio

In [4]:
serie = "PD04640PD"
estructura = f"https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{serie}/json"

data = requests.get(estructura).json()

In [5]:
## Convertir a dataframe
df = pd.DataFrame(data["periods"])
df["values"] = df["values"].apply(lambda x: x[0])
df.head()


,name,values
0,21.Feb.25,3.683
1,24.Feb.25,3.689
2,25.Feb.25,3.686
3,26.Feb.25,3.681
4,27.Feb.25,3.674


In [7]:
# Diccionario para traducir meses
meses = {
    'Ene': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Abr': 'Apr',
    'May': 'May', 'Jun': 'Jun', 'Jul': 'Jul', 'Ago': 'Aug',
    'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dic': 'Dec'
}

# Reemplazar en la columna
df['name'] = df['name'].replace(meses, regex=True)

# Convertir a datetime
df['name'] = pd.to_datetime(df['name'], format='%d.%b.%y')
df.head()


,name,values
0,2025-02-21,3.683
1,2025-02-24,3.689
2,2025-02-25,3.686
3,2025-02-26,3.681
4,2025-02-27,3.674


In [10]:
def calcular_metricas_calidad(df):
    metricas = {}

    # Asegurarse de que 'values' sea numérica
    df["values"] = pd.to_numeric(df["values"], errors="coerce")

    # Completitud: % de valores no nulos
    completitud = df["values"].notnull().mean()
    metricas["completitud"] = completitud

    # Duplicados: % de filas duplicadas
    duplicados = df.duplicated().mean()
    metricas["duplicados (%)"] = duplicados

    # Valores nulos: % de nulos en 'values'
    nulos = df["values"].isnull().mean()
    metricas["valores_nulos (%)"] = nulos

    # Validez: valores numéricos y positivos
    tasas_validas = df["values"].apply(lambda x: x > 0 if pd.notnull(x) else False).mean()
    metricas["validez (%)"] = tasas_validas

    # Consistencia: fechas ordenadas y únicas
    try:
        df["name"] = pd.to_datetime(df["name"], errors="coerce")
        fechas_ordenadas = df["name"].is_monotonic_increasing
        fechas_unicas = df["name"].is_unique
    except:
        fechas_ordenadas = False
        fechas_unicas = False
    metricas["fechas_ordenadas"] = fechas_ordenadas
    metricas["fechas_unicas"] = fechas_unicas

    # Precisión: ¿tienen exactamente 3 decimales?
    precision = df["values"].dropna().apply(lambda x: len(str(x).split('.')[-1]) == 3).mean()
    metricas["precision_3_decimales (%)"] = precision

    # Timeliness: ¿última fecha registrada qué tan reciente es?
    if df["name"].notnull().any():
        dias_desde_ultima_fecha = (pd.Timestamp.today() - df["name"].max()).days
    else:
        dias_desde_ultima_fecha = np.nan
    metricas["dias_desde_ultima_fecha"] = dias_desde_ultima_fecha

    # Conformidad: formato de fecha válido
    formatos_validos = df["name"].notnull().mean()
    metricas["formato_fecha_valido (%)"] = formatos_validos

    # Imprimir todo
    for k, v in metricas.items():
        print(f"{k}: {v}")

    return metricas

In [11]:
calcular_metricas_calidad(df)

completitud: 1.0
duplicados (%): 0.0
valores_nulos (%): 0.0
validez (%): 1.0
fechas_ordenadas: True
fechas_unicas: True
precision_3_decimales (%): 0.9
dias_desde_ultima_fecha: 1
formato_fecha_valido (%): 1.0


{'completitud': np.float64(1.0),
 'duplicados (%)': np.float64(0.0),
 'valores_nulos (%)': np.float64(0.0),
 'validez (%)': np.float64(1.0),
 'fechas_ordenadas': True,
 'fechas_unicas': True,
 'precision_3_decimales (%)': np.float64(0.9),
 'dias_desde_ultima_fecha': 1,
 'formato_fecha_valido (%)': np.float64(1.0)}